##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텐서플로 허브와 전이학습

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도
불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다.
이 번역에 개선할 부분이 있다면
[tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n/) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다.
문서 번역이나 리뷰에 참여하려면
[docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로
메일을 보내주시기 바랍니다.

[텐서플로 허브](http://tensorflow.org/hub)텐서플로 허브는 이 전에 학습된 모델들의 요소들을 공유하는 하나의 방법입니다. 학습된 모델들의 검색가능한 리스트에 대한 [텐서플로 모듈 허브] (https://tfhub.dev/)를 보세요. 이 튜토리얼은 입증합니다:

1. `tf.keras`로 텐서플로 허브를 사용하는 방법.
1. 간단한 전이학습을 하는 방법.

## 설치하기


In [1]:
import matplotlib.pylab as plt

import tensorflow as tf
import numpy as np
#import PIL.Image as Image

In [ ]:
!pip install -U tf-hub-nightly
import tensorflow_hub as hub

from tensorflow.keras import layers

### 분류기 다운로드하기

이동 네트워크 컴퓨터를 로드하기 위해 `hub.module`을, 그리고 하나의 keras층으로 감싸기 위해 `tf.keras.layers.Lambda`를 사용하세요. Fthub.dev의 [텐서플로2.0 버전의 양립 가능한 이미지 분류기 URL](https://tfhub.dev/s?q=tf2&module-type=image-classification) 는 이곳에서 작동할 것입니다.

In [3]:
# mobilenet_v2 사용. 1001 클래스 분류
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [4]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

## 간단한 전이 학습


### 데이터셋

 텐서플로의 flowers 데이터셋을 사용, 클래스 5개, 총 샘플수 3670

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

우리의 모델에 이 데이터를 가장 간단하게 로딩 하는 방법은 `tf.keras.preprocessing.image.image.ImageDataGenerator`를 사용



In [ ]:
# 0~1 사이의 값으로 정규화
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE)

결과로 나온 오브젝트는 `image_batch`와 `label_batch`를 같이 리턴 하는 반복자입니다.

In [ ]:
# image_data: DirectoryIterator, (x,y) x:(배치크기,이미지사이즈,채널), y:(배치크기,레이블)
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

### 이미지 배치에 대한 분류기를 실행해보자


이제 이미지 배치에 대한 분류기를 실행해봅시다.

In [ ]:
result_batch = classifier.predict(image_batch)
result_batch.shape

In [ ]:
# 레이블 정보를 ImageNet 데이터셋 레이블에서 가져오기
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
# 배치에 들어있는 32개 데이터 레이블 예측 (ImageNet 데이터셋 레이블 이용)
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

얼마나 많은 예측들이 이미지에 맞는지 검토해봅시다:

In [ ]:
# 32개 중, 20개만 예측
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(20):
  plt.subplot(4,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

결과가 완벽과는 거리가 멀지만, 모델이 ("daisy"를 제외한) 모든 것을 대비해서 학습된 클래스가 아니라는 것을 고려하면 합리적입니다.

### 헤드리스 모델을 다운로드하세요

텐서플로 허브는 맨 위 분류층이 없어도 모델을 분배 시킬 수 있습니다. 이는 전이 학습을 쉽게 할 수 있게 만들었습니다. -> 분류를 위한 마지막 층이 없는 모델 다운로드

fthub.dev의 [텐서플로 2.0버전의 양립 가능한 이미지 특징 벡터 URL](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2) 은 모두 이 곳에서 작동할 것입니다.

In [16]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

특성 추출기를 만들어봅시다.  

In [17]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))

이 것은 각각의 이미지마다 길이가 1280인 벡터가 반환됩니다:

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

특성 추출기 계층에 있는 변수들을 굳히면, 학습은 오직 새로운 분류 계층만 변경시킬 수 있습니다.

In [34]:
# False의 의미: 마지막 분류하는 classifier만 새로 학습
feature_extractor_layer.trainable = False   # True로 하면 전체 네트워크 파라미터 fine-tuning

### 분류 head를 붙이세요.

이제 `tf.keras.Sequential` 모델에 있는 허브 계층을 포장하고, 새로운 분류 계층을 추가하세요. -> 분류를 위한 마지막 레이어 추가

In [ ]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes, activation='softmax')
])

model.summary()

In [36]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### 모델을 학습시키세요

학습 과정 환경을 설정하기 위해 컴파일을 사용하세요:

In [38]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

이제 모델을 학습시키기 위해 `.fit`방법을 사용하세요.

예제를 짧게 유지시키기 위해 오로지 1epochs만 학습시키세요. 학습 과정을 시각화하기 위해서, 맞춤형 회신을 사용하면 손실과, 세대 평균이 아닌 배치 개별의 정확도를 기록할 수 있습니다.

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()


steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data, epochs=1,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback])

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [ ]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### 예측을 확인하세요

이 전의 계획을 다시하기 위해서, 클래스 이름들의 정렬된 리스트를 첫번째로 얻으세요:

In [ ]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

모델을 통해 이미지 배치를 실행시키세요. 그리고 인덱스들을 클래스 이름으로 바꾸세요.

In [44]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

In [45]:
label_id = np.argmax(label_batch, axis=-1)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## 당신의 모델을 내보내세요

당신은 모델을 학습시켜왔기 때문에, 저장된 모델을 내보내세요:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# SavedModel로 전체 모델을 저장합니다. 코랩 쓰는 경우, 드라이브 마운트 후, 경로 설정
!mkdir -p saved_model
model.save('/content/drive/MyDrive/saved_model/my_model')  

이제 우리는 그것을 새롭게 로딩 할 수 있고, 이는 같은 결과를 줄 것입니다:

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')  #Colab 쓰는 경우, 드라이브 마운트 후, 경로 설정

# 모델 구조를 확인합니다
new_model.summary()

In [51]:
result_batch = model.predict(image_batch)
reloaded_result_batch = new_model.predict(image_batch)

In [ ]:
abs(reloaded_result_batch - result_batch).max()